# Sample for KFService Explainer

The notebook shows how to use KFServing SDK to create KFService with explainer, predictor.

In [ ]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2ExplainerSpec
from kfserving import V1alpha2AlibiExplainerSpec
from kfserving import V1alpha2SKLearnSpec
from kfserving import V1alpha2KFServiceSpec
from kfserving import V1alpha2KFService
from kubernetes.client import V1ResourceRequirements

## Define KFService with Explainer

Add predictor and explainer on the endpoint spec

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            sklearn=V1alpha2SKLearnSpec(
                              storage_uri='gs://seldon-models/sklearn/moviesentiment',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))),
                          explainer=V1alpha2ExplainerSpec(
                            min_replicas=1, 
                            alibi=V1alpha2AlibiExplainerSpec(
                              type='AnchorText',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'2Gi'},
                                  limits={'cpu':'100m', 'memory':'2Gi'}))))
    
kfsvc = V1alpha2KFService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='moviesentiment', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_endpoint_spec))

## Create KFService

Call KFServingClient to create KFService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(kfsvc)

## Check the KFService

In [ ]:
KFServing.get('moviesentiment', namespace='kubeflow', watch=True, timeout_seconds=120)

## Predict the sentiment for the sentence

In [ ]:
CLUSTER_IP=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'

In [ ]:
!MODEL_NAME=moviesentiment
%env CLUSTER_IP=$CLUSTER_IP

In [ ]:
import os
url = "http://" + os.environ['CLUSTER_IP'].strip("[]''") + "/v1/models/moviesentiment:predict"
headers = { 'Host': 'moviesentiment-predict.kubeflow.example.com' }
import requests 
response = requests.post(url, '{"instances":["This is a bad book ."]}', headers=headers)
response.content

## Explain the Prediction

In [ ]:
import os
url = "http://" + os.environ['CLUSTER_IP'].strip("[]''") + "/v1/models/moviesentiment:explain"
headers = { 'Host': 'moviesentiment-explain.kubeflow.example.com' }
import requests 
response = requests.post(url, '{"instances":["This is a bad book ."]}', headers=headers)
response.content

## Delete the KFService

In [ ]:
KFServing.delete('moviesentiment', namespace='kubeflow')